In [29]:
# Imports

# Data tools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# scikit learn processing tools
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score

# scitkit learn models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# from xgboost.sklearn.XGBModel import XGBClassifier
from xgboost import XGBClassifier

# scikit learn metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve

# Imbalanced Learn
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline

# misc
from pprint import pprint

In [58]:
# Set options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline

In [31]:
# Binary confusion matrices are consturcted as follows:
# [[tn, fp],
#  [fn, tp]]

In [32]:
def print_scores(estimator, estimator_title, xtr, xtst, ytr, ytst):
    print('{} Score train:'.format(estimator_title), estimator.score(xtr, ytr))
    print('{} Score test:'.format(estimator_title), estimator.score(xtst, ytst))
    print(classification_report(ytst, estimator.predict(xtst)))

In [33]:
# Read in data set
tc = pd.read_csv('../data/processed/telecom_churn_cleaned.csv')
tc.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,408,415,510,churn
0,128,0.0,1.0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,107,0.0,1.0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,137,0.0,0.0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,84,1.0,0.0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,75,1.0,0.0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [34]:
tc.describe()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,408,415,510,churn
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,0.096910,0.276628,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856,0.015602,0.024002,0.016502,0.019202,0.010201,0.019802,0.022202,0.016202,0.018302,0.018902,0.016202,0.015902,0.013201,0.021902,0.017402,0.021302,0.021002,0.017702,0.015302,0.019502,0.021002,0.018602,0.021902,0.025203,0.018902,0.019502,0.020402,0.020402,0.018602,0.018302,0.016802,0.020402,0.018602,0.019802,0.024902,0.023402,0.018302,0.023402,0.013501,0.019502,0.018002,0.018002,0.015902,0.021602,0.021602,0.023102,0.021902,0.019802,0.023402,0.031803,0.023102,0.251425,0.496550,0.252025,0.144914
std,39.822106,0.295879,0.447398,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491,0.123947,0.153079,0.127414,0.137255,0.100499,0.139340,0.147363,0.126269,0.134061,0.136199,0.126269,0.125114,0.114153,0.146386,0.130782,0.144411,0.143413,0.131885,0.122768,0.138302,0.143413,0.135134,0.146386,0.156763,0.136199,0.138302,0.141392,0.141392,0.135134,0.134061,0.128547,0.141392,0.135134,0.139340,0.155851,0.151200,0.134061,0.151200,0.115426,0.138302,0.132978,0.132978,0.125114,0.145402,0.145402,0.150251,0.146386,0.139340,0.151200,0.175502,0.150251,0.433897,0.500063,0.434241,0.352067
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,74.000000,0.000000,0.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50

In [35]:
tc = tc.drop('CA', axis=1)
tc.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,AK,AL,AR,AZ,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,408,415,510,churn
0,128,0.0,1.0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,107,0.0,1.0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,137,0.0,0.0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,84,1.0,0.0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,75,1.0,0.0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [36]:
# Set up train/test split, create some extra copies to investigate scaling
y_orig = tc['churn'].values.copy()
X_orig = tc.drop('churn', axis=1).copy()
X, X_holdout, y, y_holdout = train_test_split(X_orig, y_orig, test_size=0.2,
                                                          random_state=42, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                          random_state=42, shuffle=True)
X_train.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,AK,AL,AR,AZ,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,408,415,510
42,34,0.0,0.0,0,124.8,82,21.22,282.2,98,23.99,311.5,78,14.02,10.0,4,2.70,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1166,72,0.0,0.0,0,118.2,106,20.09,167.2,136,14.21,214.2,106,9.64,12.2,3,3.29,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1895,86,0.0,1.0,21,197.9,99,33.64,165.6,100,14.08,208.0,120,9.36,10.1,9,2.73,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1083,82,1.0,0.0,0,208.8,101,35.50,213.7,87,18.16,175.1,86,7.88,12.4,6,3.35,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
129,131,0.0,1.0,36,214.2,115,36.41,161.7,117,13.74,264.7,102,11.91,9.5,4,2.57,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### Steps to test for each model
1. StandardScaler fit/predict
2. Variations on hyperparameters
3. Balanced dataset fit predict (Random/Smote/Adyson)
4. Variations on hypterparameters with balanced data

In [37]:
steps = [('scaler', StandardScaler()),
         ('LR', LogisticRegression(solver='lbfgs', max_iter=1000))]

pipeline = Pipeline(steps)

parameters = {'LR__C' : [12000, 11000, 10000, 9000, 8000]}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5)
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(cv.score(X_test, y_test)))
print("Accuracy Test: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy Train: 0.8670411985018727
Accuracy Test: 0.8670411985018727
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       456
           1       0.62      0.23      0.34        78

    accuracy                           0.87       534
   macro avg       0.75      0.60      0.63       534
weighted avg       0.84      0.87      0.84       534

Tuned Model Parameters: {'LR__C': 10000}


#### Logistic Regression continually gets the best score at C=10000. Using C=10000 for further testing.  
#### Next step is try over sampling with Logistic Regression.

In [10]:
# Use Random Oversamping to see if score improves
steps = [('OR', RandomOverSampler(sampling_strategy='minority',random_state=42)),
         ('scaler', StandardScaler()),
         ('LR', LogisticRegression(solver='lbfgs', C=10000, max_iter=1000))]

lr_pipe = Pipeline(steps)

cross_val_score(lr_pipe, X_train, y_train, cv=5, n_jobs=-1)

# lr_pipe.fit(X, y)
# y_pred = lr_pipe.predict(X_test)

# # Compute and print metrics
# print("Accuracy Train: {}".format(lr_pipe.score(X_test, y_test)))
# print("Accuracy Test: {}".format(lr_pipe.score(X_test, y_test)))
# print(classification_report(y_test, y_pred))

array([0.76580796, 0.76112412, 0.75175644, 0.75117371, 0.75058824])

#### Random Oversampling decreased score. Trying ADASYN.

In [11]:
# Use Adasyn Oversamping to see if score improves
steps = [('ADASYN', ADASYN(sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('LR', LogisticRegression(solver='lbfgs', C=10000, max_iter=1000))]

lr_pipe = Pipeline(steps)
cross_val_score(lr_pipe, X_train, y_train, cv=5, n_jobs=-1)

# lr_pipe.fit(X, y)
# y_pred = lr_pipe.predict(X_test)

# # Compute and print metrics
# print("Accuracy Train: {}".format(lr_pipe.score(X_test, y_test)))
# print("Accuracy Test: {}".format(lr_pipe.score(X_test, y_test)))
# print(classification_report(y_test, y_pred))

array([0.77283372, 0.74238876, 0.73067916, 0.72769953, 0.74352941])

#### ADASYN performed worse on average than Random Oversampling.  
#### Trying SMOTE.

In [12]:
# Use SMOTE Oversamping to see if score improves
steps = [('SMOTE', SMOTE(sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('LR', LogisticRegression(solver='lbfgs', C=10000, max_iter=1000))]

lr_pipe = Pipeline(steps)
cross_val_score(lr_pipe, X_train, y_train, cv=5, n_jobs=-1)

# lr_pipe.fit(X, y)
# y_pred = lr_pipe.predict(X_test)

# # Compute and print metrics
# print("Accuracy Train: {}".format(lr_pipe.score(X_test, y_test)))
# print("Accuracy Test: {}".format(lr_pipe.score(X_test, y_test)))
# print(classification_report(y_test, y_pred))

array([0.79156909, 0.7587822 , 0.75175644, 0.74882629, 0.75764706])

#### SMOTE comes in between Random Oversampling and ADASYN.  
#### Trying SMOTENC

In [13]:
# This will create a list of indexes for feature (column) in the dataframe that is categorical.
X.AK.unique().shape[0]

categoricals = []
for idx, val in enumerate(X.columns):
    if X[val].unique().shape[0] == 2:
        categoricals.append(idx)
# categoricals

In [14]:
# Use SMOTENC Oversamping to see if score improves
steps = [('SMOTENC', SMOTENC(categoricals, sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('LR', LogisticRegression(solver='lbfgs', C=10000, max_iter=1000))]

lr_pipe = Pipeline(steps)
cross_val_score(lr_pipe, X_train, y_train, cv=5, n_jobs=-1)

# lr_pipe.fit(X, y)
# y_pred = lr_pipe.predict(X_test)

# # Compute and print metrics
# print("Accuracy Train: {}".format(lr_pipe.score(X_test, y_test)))
# print("Accuracy Test: {}".format(lr_pipe.score(X_test, y_test)))
# print(classification_report(y_test, y_pred))

array([0.8618267 , 0.8618267 , 0.83138173, 0.85915493, 0.86352941])

### Performance of Logistic Regression is best without oversampling.  
### Start testing with Naive Bayes

In [15]:


steps = [('scaler', StandardScaler()),
         ('NB', GaussianNB())]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

Accuracy Train: 0.5955056179775281
Accuracy Test: 0.5955056179775281
              precision    recall  f1-score   support

           0       0.90      0.59      0.72       456
           1       0.20      0.60      0.30        78

    accuracy                           0.60       534
   macro avg       0.55      0.60      0.51       534
weighted avg       0.80      0.60      0.65       534



In [16]:
# Use Random Oversamping to see if score improves
steps = [('OR', RandomOverSampler(sampling_strategy='minority',random_state=42)),
         ('scaler', StandardScaler()),
         ('NB', GaussianNB())]

nb_pipe = Pipeline(steps)

cross_val_score(nb_pipe, X_train, y_train, cv=5, n_jobs=-1)

array([0.52459016, 0.56674473, 0.44730679, 0.33333333, 0.52235294])

In [17]:
def get_categoricals(frame):
    categoricals = []
    for idx, val in enumerate(frame.columns):
        if frame[val].nunique() == 2:
            categoricals.append(idx)
    return categoricals
# get_categoricals(X_train)

In [18]:
def get_discrete_cont(frame, labels=True):
    discrete_cont = []
    for idx, val in enumerate(frame.columns):
        if frame[val].nunique() > 50:
            if labels:
                discrete_cont.append(val)
            else:
                discrete_cont.append(idx)
    return discrete_cont
# get_discrete_cont(X_train, True)

In [19]:
def apply_cut(frame, cuts=25, labels=False):
    new_frame = frame.copy()
    discrete_cont = get_discrete_cont(frame, labels=True)
    for val in discrete_cont:
        new_frame[val] = pd.qcut(new_frame[val], cuts, labels=labels)
    return new_frame
# apply_qcut(X_train).head(3)

In [20]:
C = apply_cut(X)
C_train, C_test, d_train, d_test = train_test_split(C, y)
C.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,AK,AL,AR,AZ,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,408,415,510
817,24,0.0,0.0,0,1,7,1,5,0,5,22,20,22,2,6,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1373,14,0.0,0.0,0,2,14,2,11,17,11,14,8,13,0,4,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
679,6,1.0,0.0,0,19,3,19,24,17,24,13,14,13,6,9,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
56,21,0.0,0.0,0,4,11,4,8,0,8,2,23,2,4,2,4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1993,8,0.0,0.0,0,18,9,18,22,2,22,14,17,14,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [21]:
steps = [('NB', MultinomialNB())]

pipeline = Pipeline(steps)
pipeline.fit(C_train, d_train)

# Predict the labels of the test set: y_pred
d_pred = pipeline.predict(C_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(C_test, d_test)))
print("Accuracy Test: {}".format(pipeline.score(C_test, d_test)))
print(classification_report(d_test, d_pred))

Accuracy Train: 0.5247376311844077
Accuracy Test: 0.5247376311844077
              precision    recall  f1-score   support

           0       0.90      0.50      0.64       572
           1       0.18      0.68      0.29        95

    accuracy                           0.52       667
   macro avg       0.54      0.59      0.47       667
weighted avg       0.80      0.52      0.59       667



In [22]:
steps = [('OR', RandomOverSampler(sampling_strategy='minority',random_state=42)),
         ('NB', MultinomialNB())]

pipeline = Pipeline(steps)
pipeline.fit(C_train, d_train)

# Predict the labels of the test set: y_pred
d_pred = pipeline.predict(C_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(C_test, d_test)))
print("Accuracy Test: {}".format(pipeline.score(C_test, d_test)))
print(classification_report(d_test, d_pred))

Accuracy Train: 0.44677661169415295
Accuracy Test: 0.44677661169415295
              precision    recall  f1-score   support

           0       0.90      0.40      0.55       572
           1       0.17      0.75      0.28        95

    accuracy                           0.45       667
   macro avg       0.54      0.57      0.41       667
weighted avg       0.80      0.45      0.51       667



In [23]:
steps = [('ADASYN', ADASYN(sampling_strategy='minority', random_state=42)),
         ('NB', MultinomialNB())]

pipeline = Pipeline(steps)
pipeline.fit(C_train, d_train)

# Predict the labels of the test set: y_pred
d_pred = pipeline.predict(C_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(C_test, d_test)))
print("Accuracy Test: {}".format(pipeline.score(C_test, d_test)))
print(classification_report(d_test, d_pred))

Accuracy Train: 0.44377811094452774
Accuracy Test: 0.44377811094452774
              precision    recall  f1-score   support

           0       0.91      0.39      0.55       572
           1       0.17      0.77      0.28        95

    accuracy                           0.44       667
   macro avg       0.54      0.58      0.41       667
weighted avg       0.81      0.44      0.51       667



In [24]:
steps = [('SMOTE', SMOTE(sampling_strategy='minority', random_state=42)),
         ('NB', MultinomialNB())]

pipeline = Pipeline(steps)
pipeline.fit(C_train, d_train)

# Predict the labels of the test set: y_pred
d_pred = pipeline.predict(C_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(C_test, d_test)))
print("Accuracy Test: {}".format(pipeline.score(C_test, d_test)))
print(classification_report(d_test, d_pred))

Accuracy Train: 0.4362818590704648
Accuracy Test: 0.4362818590704648
              precision    recall  f1-score   support

           0       0.90      0.38      0.54       572
           1       0.17      0.76      0.28        95

    accuracy                           0.44       667
   macro avg       0.54      0.57      0.41       667
weighted avg       0.80      0.44      0.50       667



In [25]:
categoricals = list(range(0, len(C_train.columns)))

In [26]:
(C_train.shape, d_train.shape)

((1999, 70), (1999,))

In [27]:
steps = [('SMOTENC', SMOTENC(categoricals[1:], sampling_strategy='minority', random_state=42)),
         ('NB', MultinomialNB())]

pipeline = Pipeline(steps)
pipeline.fit(C_train, d_train)

# Predict the labels of the test set: y_pred
d_pred = pipeline.predict(C_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(C_test, d_test)))
print("Accuracy Test: {}".format(pipeline.score(C_test, d_test)))
print(classification_report(d_test, d_pred))

Accuracy Train: 0.41829085457271364
Accuracy Test: 0.41829085457271364
              precision    recall  f1-score   support

           0       0.90      0.36      0.52       572
           1       0.16      0.76      0.27        95

    accuracy                           0.42       667
   macro avg       0.53      0.56      0.39       667
weighted avg       0.80      0.42      0.48       667



### Support Vector Classifiers unbalanced and balanced

In [28]:
svc = SVC(kernel='linear', class_weight='balanced')
_ = svc.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# steps = [('scaler', StandardScaler()),
#          ('SVC', SVC(kernel='linear', class_weight='balanced'))]
steps = [('scaler', StandardScaler()),
         ('SVC', SVC())]

pipeline = Pipeline(steps)

parameters = [{'SVC__C' : [1, 10, 100, 1000],
              'SVC__kernel' : ['rbf'],
              'SVC__gamma': [1e-3, 1e-4]},
              {'SVC__C' : [1, 10, 100, 1000],
              'SVC__kernel' : ['linear']}]

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, n_jobs=-1)
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(cv.score(X_test, y_test)))
print("Accuracy Test: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

### SVC with oversampled datasets

In [ ]:
steps = [('OR', RandomOverSampler(sampling_strategy='minority',random_state=42)),
         ('scaler', StandardScaler()),
         ('SVC', SVC(C=100, gamma=0.001, kernel='rbf'))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
steps = [('SMOTE', SMOTE(sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('SVC', SVC(C=100, gamma=0.001, kernel='rbf'))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
categoricals = get_categoricals(X_train)

steps = [('SMOTENC', SMOTENC(categoricals, sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('SVC', SVC(C=100, gamma=0.001, kernel='rbf'))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
steps = [('OR', RandomOverSampler(sampling_strategy='minority',random_state=42)),
         ('scaler', StandardScaler()),
         ('SVC', SVC(C=100, gamma=0.001, kernel='rbf'))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

### KNN Classifier

In [ ]:
knn = KNeighborsClassifier()
_ = knn.fit(X_train, y_train)

In [ ]:
steps = [('scaler', StandardScaler()),
         ('KNN', KNeighborsClassifier())]

pipeline = Pipeline(steps)

parameters = {'KNN__n_neighbors' : [10, 9, 8, 7, 6, 5, 4, 3, 2]}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5)
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(cv.score(X_test, y_test)))
print("Accuracy Test: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

### Random Forest Classifier

In [ ]:
X_old = X_train
y_old = y_train
X_train.head()

In [ ]:
steps = [('scaler', StandardScaler()),
         ('RF', RandomForestClassifier(random_state=42))]

pipeline = Pipeline(steps)

parameters = {'RF__n_estimators' : [300, 325, 350, 375, 400]}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5)
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(cv.score(X_test, y_test)))
print("Accuracy Test: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

In [ ]:
steps = [('scaler', StandardScaler()),
         ('RF', RandomForestClassifier(n_estimators=300, random_state=42))]

pipeline = Pipeline(steps)
cv = pipeline
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(cv.score(X_test, y_test)))
print("Accuracy Test: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
steps = [('OR', RandomOverSampler(sampling_strategy='minority',random_state=42)),
#          ('scaler', StandardScaler()),
         ('RF', RandomForestClassifier(n_estimators=400, random_state=42))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
plt.figure(figsize=(12, 5))
fpr, tpr, thresholds = roc_curve(y_test, pipeline.predict_proba(X_test)[:,1])
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for Customer Churn');
print("ROC AUC score = ", roc_auc_score(y_test, pipeline.predict_proba(X_test)[:,1]))

plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, pipeline.predict(X_test)), cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['0', '1'],
           yticklabels=['0', '1']);
plt.xlabel('prediction')
plt.ylabel('actual');

In [ ]:
steps = [('ADASYN', ADASYN(sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('RF', RandomForestClassifier(n_estimators=400, random_state=42))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
plt.figure(figsize=(12, 5))
fpr, tpr, thresholds = roc_curve(y_test, cv.predict_proba(X_test)[:,1])
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for Customer Churn');
print("ROC AUC score = ", roc_auc_score(y_test, cv.predict_proba(X_test)[:,1]))

plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, cv.predict(X_test)), cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['0', '1'],
           yticklabels=['0', '1']);
plt.xlabel('prediction')
plt.ylabel('actual');

In [ ]:
steps = [('SMOTE', SMOTE(sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('RF', RandomForestClassifier(n_estimators=400, random_state=42))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
categoricals = get_categoricals(X_train)

steps = [('SMOTENC', SMOTENC(categoricals, sampling_strategy='minority', random_state=42)),
         ('scaler', StandardScaler()),
         ('RF', RandomForestClassifier(n_estimators=400, random_state=42))]

pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = pipeline.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(pipeline.score(X_test, y_test)))
print("Accuracy Test: {}".format(pipeline.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
xg = XGBClassifier(n_estimators=300)

In [ ]:
xg = XGBClassifier(n_estimators=200, random_state=42)
xg.fit(X_train, y_train)
y_pred = xg.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(xg.score(X_test, y_test)))
print("Accuracy Test: {}".format(xg.score(X_test, y_test)))
print(classification_report(y_test, y_pred))

In [ ]:
plt.figure(figsize=(12, 5))
fpr, tpr, thresholds = roc_curve(y_test, xg.predict_proba(X_test)[:,1])
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for Customer Churn');
print("ROC AUC score = ", roc_auc_score(y_test, xg.predict_proba(X_test)[:,1]))

plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, xg.predict(X_test)), cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['0', '1'],
           yticklabels=['0', '1']);
plt.xlabel('prediction')
plt.ylabel('actual');

In [ ]:
steps = [('XG', XGBClassifier(random_state=42))]

pipeline = Pipeline(steps)

parameters = {'XG__n_estimators' : [200, 225, 250, 275],
              'XG__max_depth' : [6]}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5)
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy Train: {}".format(cv.score(X_test, y_test)))
print("Accuracy Test: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

### Best Results are XGBoost, 250 learners, max depth of 6  
Accuracy Train: 0.9644194756554307  
Accuracy Test: 0.9644194756554307  
              precision    recall  f1-score   support  

           0       0.97      0.99      0.98       456  
           1       0.93      0.82      0.87        78  

    accuracy                           0.96       534  
   macro avg       0.95      0.90      0.93       534  
weighted avg       0.96      0.96      0.96       534  
  
Tuned Model Parameters: {'XG__max_depth': 6, 'XG__n_estimators': 250}  

In [ ]:
plt.figure(figsize=(12, 5))
fpr, tpr, thresholds = roc_curve(y_test, cv.predict_proba(X_test)[:,1])
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for Customer Churn');
print("ROC AUC score = ", roc_auc_score(y_test, cv.predict_proba(X_test)[:,1]))

plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, cv.predict(X_test)), cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['0', '1'],
           yticklabels=['0', '1']);
plt.xlabel('prediction')
plt.ylabel('actual');

In [ ]:
xg = XGBClassifier(random_state=42, n_estimators=250, max_depth=6)
xg.fit(X_train, y_train)

In [ ]:
xg.feature_importances_

In [ ]:
def get_feature_names(dataset, importances):
    important_features = []
    feature_values = []
    for idx, val in enumerate(importances):
        if val > 0.01:
            important_features.append(True)
        else:
            important_features.append(False)
        feature_values.append(val)
    important_features = np.array(important_features)
    feature_names = np.array(dataset.columns)[important_features]
    feature_values = np.array(feature_values)[important_features]
    value_sort = np.argsort(feature_values)
    
    return (feature_names[value_sort]., feature_values[value_sort])
        

In [ ]:
a, b = get_feature_names(X_train, xg.feature_importances_)


In [ ]:
(a[::-1], b[::-1])

In [51]:
xg = XGBClassifier(n_estimators=250, max_depth=6, random_state=42)

xg.fit(X_train.values, y_train)

# Predict the labels of the test set: y_pred
y_pred = xg.predict(X_test.values)

# Compute and print metrics
print("Accuracy Train: {}".format(xg.score(X_test.values, y_test)))
print("Accuracy Test: {}".format(xg.score(X_test.values, y_test)))
print(classification_report(y_test, y_pred))

Accuracy Train: 0.9644194756554307
Accuracy Test: 0.9644194756554307
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       456
           1       0.93      0.82      0.87        78

    accuracy                           0.96       534
   macro avg       0.95      0.90      0.93       534
weighted avg       0.96      0.96      0.96       534



In [40]:
import pickle

In [52]:
with open('xgboost.pkl', 'wb') as f:
    pickle.dump(xg, f)

In [56]:
X_test.values[0].reshape(1, -1)[0].shape

(70,)

In [59]:
X_test.iloc[0]

account length             87.00
international plan          0.00
voice mail plan             1.00
number vmail messages      28.00
total day minutes         151.40
total day calls            95.00
total day charge           25.74
total eve minutes         152.40
total eve calls            97.00
total eve charge           12.95
total night minutes       250.10
total night calls         109.00
total night charge         11.25
total intl minutes          0.00
total intl calls            0.00
total intl charge           0.00
customer service calls      1.00
AK                          0.00
AL                          0.00
AR                          0.00
AZ                          0.00
CO                          0.00
CT                          0.00
DC                          0.00
DE                          0.00
FL                          0.00
GA                          0.00
HI                          1.00
IA                          0.00
ID                          0.00
IL        

In [ ]:
{'account_length': 107,
 'international_plan': 0,
 'voice_mail_plan': 1,
 'number_vmail_messages': 26,
 'total_day_minutes': 161.6,
 'total_day_calls': 123,
 'total_day_charge': 27.47,
 'total_eve_minutes': 195.5,
 'total_eve_calls': 103,
 'total_eve_charge': 16.62,
 'total_night_minutes': 254.4,
 'total_night_calls': 103,
 'total_night_charge': 11.45,
 'total_intl_minutes': 13.7,
 'total_intl_calls': 3,
 'total_intl_charge': 3.7,
 'customer_service_calls': 1,
 'AK': 0,
 'AL': 0,
 'AR': 0,
 'AZ': 0,
 'CA': 0,
 'CO': 0,
 'CT': 0,
 'DC': 0,
 'DE': 0,
 'FL': 0,
 'GA': 0,
 'HI': 0,
 'IA': 0,
 'ID': 0,
 'IL': 0,
 'IN': 0,
 'KS': 0,
 'KY': 0,
 'LA': 0,
 'MA': 0,
 'MD': 0,
 'ME': 0,
 'MI': 0,
 'MN': 0,
 'MO': 0,
 'MS': 0,
 'MT': 0,
 'NC': 0,
 'ND': 0,
 'NE': 0,
 'NH': 0,
 'NJ': 0,
 'NM': 0,
 'NV': 0,
 'NY': 0,
 'OH': 1,
 'OK': 0,
 'OR': 0,
 'PA': 0,
 'RI': 0,
 'SC': 0,
 'SD': 0,
 'TN': 0,
 'TX': 0,
 'UT': 0,
 'VA': 0,
 'VT': 0,
 'WA': 0,
 'WI': 0,
 'WV': 0,
 'WY': 0,
 408: 0,
 415: 0,
 510: 0}


In [ ]:
account length             87.00
international plan          0.00
voice mail plan             1.00
number vmail messages      28.00
total day minutes         151.40
total day calls            95.00
total day charge           25.74
total eve minutes         152.40
total eve calls            97.00
total eve charge           12.95
total night minutes       250.10
total night calls         109.00
total night charge         11.25
total intl minutes          0.00
total intl calls            0.00
total intl charge           0.00
customer service calls      1.00
AK                          0.00
AL                          0.00
AR                          0.00
AZ                          0.00
CO                          0.00
CT                          0.00
DC                          0.00
DE                          0.00
FL                          0.00
GA                          0.00
HI                          1.00
IA                          0.00
ID                          0.00
IL                          0.00
IN                          0.00
KS                          0.00
KY                          0.00
LA                          0.00
MA                          0.00
MD                          0.00
ME                          0.00
MI                          0.00
MN                          0.00
MO                          0.00
MS                          0.00
MT                          0.00
NC                          0.00
ND                          0.00
NE                          0.00
NH                          0.00
NJ                          0.00
NM                          0.00
NV                          0.00
NY                          0.00
OH                          0.00
OK                          0.00
OR                          0.00
PA                          0.00
RI                          0.00
SC                          0.00
SD                          0.00
TN                          0.00
TX                          0.00
UT                          0.00
VA                          0.00
VT                          0.00
WA                          0.00
WI                          0.00
WV                          0.00
WY                          0.00
408                         1.00
415                         0.00
510                         0.00